In [ ]:
import pandas as pd
import geopandas as gpd

bus = pd.read_csv("../dataset/city/서울시 대중교통시설 위치정보_bus.csv", encoding="CP949")
subway = pd.read_csv("../dataset/city/서울시 대중교통시설 위치정보_sub.csv", encoding="CP949")

### 지하철 데이터 전처리

In [ ]:
subway.head()

In [ ]:
# 데이터 백틱(`) 제거

column_name = subway.columns

for i in column_name:
    subway[i] = subway[i].str[1:-1]

subway.head()

In [ ]:
# 필요 컬럼 추출
subway = subway[['`GU_NM`', '`KOR_SUB_NM`', '`POINT_X`', '`POINT_Y`']]

# 컬럼명 변경
subway = subway.rename(columns={'`GU_NM`': 'gu',
                               '`KOR_SUB_NM`': 'sub_name',
                               '`POINT_X`':'X_VALUE',
                               '`POINT_Y`':'Y_VALUE'})
subway.head()


In [ ]:
# X,Y좌표 경도 위도로 변환

sub = gpd.GeoDataFrame(subway, geometry=gpd.points_from_xy(subway.X_VALUE, subway.Y_VALUE))

sub.crs = 'epsg:5181'
sub = sub.to_crs('epsg:4326')

# 변환 좌표 속성 생성 
sub['x'] = sub['geometry'].x
sub['y'] = sub['geometry'].y

# 필요한 컬럼만 추출해서 저장하기
sub = sub[['sub_name', 'x', 'y']]

print(sub)

In [ ]:
# 경도/위도로 시도/구/행정동 단위 변환 (카카오 api)

import pandas as pd
import requests 

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in sub.iterrows():
    get_location(row)
    
    # 20개씩 처리될 때 메시지 출력
    if i % 20 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
sub['city'] = [location.get('city') for location in locations]
sub['gu'] = [location.get('gu') for location in locations]
sub['dong'] = [location.get('dong') for location in locations]


In [ ]:
# 지하철 데이터 저장
sub.to_csv('../dataset/city/서울시 지하철역 위치 정보.csv')

### 버스 데이터 전처리

In [ ]:
# 데이터 백틱(`) 제거

column_name2 = bus.columns

for i in column_name2:
    bus[i] = bus[i].str[1:-1]
bus.head()

In [ ]:
# 필요 컬럼 추출

bus = bus[['`BUS_STA_NM`', '`X_COORD`',	'`Y_COORD`']]
bus = bus.rename(columns={'`BUS_STA_NM`': 'name',
                        '`X_COORD`': 'x',
                        '`Y_COORD`': 'y'})
bus

In [ ]:
# 경도/위도로 시도/구/행정동 단위 변환 (카카오 api)

import pandas as pd
import requests

locations = []

def get_location(row):
    url = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x={}&y={}'.format(row['x'], row['y'])
    headers = {
        "Authorization": "KakaoAK 8cd64c4299a983cac6fd3edef103f134"
    }
    response = requests.get(url, headers=headers)
    
    try:
        response_json = response.json()
        place = response_json['documents'][1]
        city = place.get('region_1depth_name')
        gu = place.get('region_2depth_name')
        dong = place.get('region_3depth_name')
        locations.append({'city': city, 'gu': gu, 'dong': dong})
    except (IndexError, AttributeError, KeyError):
        print('API 호출 오류:', response.text)

# 데이터프레임의 각 행에 대해 get_location 함수 호출
for i, row in bus.iterrows():
    get_location(row)
    
    # 300개씩 처리될 때 메시지 출력
    if i % 300 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

# 결과를 데이터프레임에 추가
bus['city'] = [location.get('city') for location in locations]
bus['gu'] = [location.get('gu') for location in locations]
bus['dong'] = [location.get('dong') for location in locations]


In [ ]:
# 경기도 데이터 제거 및 csv 저장

bus = bus[bus['city'] == '서울특별시']
bus.to_csv('../dataset/city/서울시 버스 정류장 위치 정보.csv')

### 면적 당 지하철역/버스정류장 산출

In [ ]:
bus = pd.read_csv('../dataset/city/서울시 버스 정류장 위치 정보.csv', index_col='')
sub = pd.read_csv('../dataset/city/서울시 지하철역 위치 정보.csv', index_col='')

In [ ]:
# 중복 데이터 제거
bus_no_dup = bus.drop_duplicates()
sub_no_dup = sub.drop_duplicates()

In [ ]:
# 동 별 데이터
sub_new = sub_no_dup.groupby(['city', 'gu', 'dong'], as_index=False)['sub_name'].count()
bus_new = bus_no_dup.groupby(['city', 'gu', 'dong'], as_index=False)['name'].count()

In [ ]:
# 행정동별 면적 데이터 로드

area = pd.read_csv("../dataset/city/행정동별 면적.csv")

area = area.drop(area.index[0:3])                   # 필요없는 데이터 정리
area = area.reset_index(drop=True)                  # 인덱스 리셋
area = area.drop('동별(1)', axis=1)                  # 필요없는 컬럼 삭제
area = area.rename(columns={'동별(2)': 'gu',
                            '동별(3)':'dong',
                            '2022':'area_km2'})     # 컬럼명 변경

area.head()

In [ ]:
# 면적 당 지하철역 갯수 산출

sub_merge = sub_new.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
sub_merge['area_km2'] = sub_merge['area_km2'].astype(float)
sub_merge['density_sub'] = sub_merge['sub_name'] / sub_merge['area_km2']

In [ ]:
# 면적 당 버스 정류장 갯수 산출

bus_merge = bus_new.merge(area[['gu', 'dong', 'area_km2']], on=['gu','dong'], how = 'left')
bus_merge['area_km2'] = bus_merge['area_km2'].astype(float)
bus_merge['density_bus'] = bus_merge['name'] / bus_merge['area_km2']
bus_merge

In [ ]:
bus_merge = bus_merge.rename(columns={'gu': '구', 'dong': '동',
                             'name': '정류장 수',
                             'area_km2':'면적(km2)',
                             'density_bus':'면적당 정류장 수'})

bus_merge = bus_merge[['구', '동', '정류장 수', '면적(km2)', '면적당 정류장 수']]
bus_merge.to_csv('../dataset/city/도시_면적당 정류장 수_동별_2023.csv',index=False)

In [ ]:
sub_merge = sub_merge.rename(columns={'gu': '구', 'dong': '동',
                             'sub_name': '지하철역 수',
                             'area_km2':'면적(km2)',
                             'density_sub':'면적당 지하철역 수'})

sub_merge = sub_merge[['구', '동', '지하철역 수', '면적(km2)', '면적당 지하철역 수']]
sub_merge.to_csv('../dataset/city/도시_면적당 지하철역 수_동별_2023.csv',index=False)